## Burrows-Wheeler Transform

A transformada de burrows-wheeler BW de uma string T de tamanho `m` é uma string com os caracteres de T permutados. Uma permutação possível da string T é a aplitação de uma função rotação resultando no deslocamento dos caracteres da string para a direita. O resultado desta operação `m` vezes seguido da ordenação das strings em ordem lexicográfica é a matriz `BWM`.

Por exemplo, seja T="cat". A rotação desta string m=3 vezes é:

In [153]:
def rotations(t):
    # Return list of rotations of input string t
    tt = t * 2
    return [ tt[i:i+len(t)] for i in range(0, len(t)) ]

In [154]:
rotations('cat')

['cat', 'atc', 'tca']

A ordenação lexicograficamente destas strings resulta na matriz BW:

In [155]:
def bwm(t):
    # Return lexicographically sorted list of t’s rotations
    return sorted(rotations(t))

In [157]:
print('\n'.join(bwm('cat$')))

$cat
at$c
cat$
t$ca


Reference links:
    - [FM-Index Langmead lecture](https://www.youtube.com/watch?v=kvVGj5V65io)
    - [BWT-FM Index PDF](http://www.cs.jhu.edu/~langmea/resources/bwt_fm.pdf)
    

In [4]:
bwm('abaaba$')

['$abaaba', 'a$abaab', 'aaba$ab', 'aba$aba', 'abaaba$', 'ba$abaa', 'baaba$a']

In [5]:
print('\n'.join(bwm('abaaba$')))

$abaaba
a$abaab
aaba$ab
aba$aba
abaaba$
ba$abaa
baaba$a


In [6]:
def bwtViaBwm(t):
    # Given T, returns BWT(T) by way of the BWM
    return ''.join(map(lambda x: x[-1], bwm(t)))

In [7]:
bwtViaBwm('abaaba$') # we can see the result equals the last column of the matrix above

'abba$aa'

In [196]:
def suffixArrayAll(s):
    satups = sorted([(s[i:], i) for i in xrange(0, len(s))])
    return satups

def suffixArray(s):
    satups = sorted([(s[i:], i) for i in xrange(0, len(s))])
    print satups
    return map(lambda x: x[1], satups)

def bwtViaSa(t):
    # Given T, returns BWT(T) by way of the suffix array
    bw = []
    for si in suffixArray(t):
        if si == 0:
            bw.append('$')
        else:
            bw.append(t[si-1])
    return ''.join(bw) # return string-ized version of list bw

In [197]:
suffixArray('abaaba$')

[('$', 6), ('a$', 5), ('aaba$', 2), ('aba$', 3), ('abaaba$', 0), ('ba$', 4), ('baaba$', 1)]


[6, 5, 2, 3, 0, 4, 1]

In [198]:
bwtViaBwm('abaaba$'), bwtViaSa('abaaba$') # same result

[('$', 6), ('a$', 5), ('aaba$', 2), ('aba$', 3), ('abaaba$', 0), ('ba$', 4), ('baaba$', 1)]


('abba$aa', 'abba$aa')

In [199]:
def rankBwt(bw):
    ''' Given BWT string bw, return parallel list of B-ranks (ranks in ascending order). Also
    returns tots: map from character to # times it appears.
    '''
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots: tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots

def firstCol(tots):
    '''
    Return map from character to the range of rows prefixed by
     the character.
    '''
    first = {}
    totc = 0
    for c, count in sorted(tots.iteritems()):
        first[c] = (totc, totc + count)
        totc += count
    return first

def reverseBwt(bw):
    '''Make T from BWT(T)'''
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    print(first)
    rowi = 0 #start in first row
    t = '$'
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t #preped to answer
        # jump to row that starts with c of same rank
        rowi = first[c][0] + ranks[rowi]
    return t

In [200]:
reverseBwt('abba$aa')

{'a': (1, 5), 'b': (5, 7), '$': (0, 1)}


'abaaba$'

In [201]:
ranks, tots = rankBwt('abba$aa')

In [202]:
ranks

[0, 0, 1, 1, 0, 2, 3]

In [203]:
tots

{'$': 1, 'a': 4, 'b': 2}

In [204]:
def rankAllBwt(bw):
    """ Given BWT string bw, returns a map of lists. Keys are characters and lists are
     cumulative # of occurences up to and including the row.
    """
    tots = {}
    rankAll = {}
    for c in bw:
        if c not in tots:
            tots[c] = 0
            rankAll[c] = []
    
    for c in bw:
        tots[c] += 1
        for c in tots.iterkeys():
            rankAll[c].append(tots[c])
    return rankAll, tots

def countMatches(bw, p):
    """
    Given BWT(T) and a pattern string p, return the number of times p occurs in T.
    """
    rankAll, tots = rankAllBwt(bw)
    first = firstCol(tots)
    if p[-1] not in first:
        return 0 # character doesnt occur in T
    l, r = first[p[-1]]
    i = len(p)-2
    while i >= 0 and r > l:
        c = p[i]
        l = first[c][0] + rankAll[c][l-1]
        r = first[c][0] + rankAll[c][r-1]
        i -= 1
    return l, r
    

In [205]:
countMatches( bwtViaSa('mississipi$'), 'iss')

[('$', 10), ('i$', 9), ('ipi$', 7), ('issipi$', 4), ('ississipi$', 1), ('mississipi$', 0), ('pi$', 8), ('sipi$', 6), ('sissipi$', 3), ('ssipi$', 5), ('ssissipi$', 2)]


(3, 5)

In [206]:
for s, t in zip(bwm('mississipi$'), suffixArrayAll('mississipi$')):
    print ("{0}\t{1}".format(s,t))

$mississipi	('$', 10)
i$mississip	('i$', 9)
ipi$mississ	('ipi$', 7)
issipi$miss	('issipi$', 4)
ississipi$m	('ississipi$', 1)
mississipi$	('mississipi$', 0)
pi$mississi	('pi$', 8)
sipi$missis	('sipi$', 6)
sissipi$mis	('sissipi$', 3)
ssipi$missi	('ssipi$', 5)
ssissipi$mi	('ssissipi$', 2)


In [213]:
def countMatchesAlex(bw, p):
    """
    Given BWT(T) and a pattern string p, return the number of times p occurs in T.
    """
    rankAll, tots = rankAllBwt(bw)
    first = firstCol(tots)
    s = 1
    e = len(bw)
    i = len(p)-1
    while i >= 0 and e > s:
        c = p[i]
        if c not in first.iterkeys():
            return 0
        s = first[c][0] + rankAll[c][s-1] 
        e = first[c][0] + rankAll[c][e-1]
        i -= 1
    return s, e

In [214]:
rankAllBwt(bwtViaSa('mississippi$'))

[('$', 11), ('i$', 10), ('ippi$', 7), ('issippi$', 4), ('ississippi$', 1), ('mississippi$', 0), ('pi$', 9), ('ppi$', 8), ('sippi$', 6), ('sissippi$', 3), ('ssippi$', 5), ('ssissippi$', 2)]


({'$': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
  'i': [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 4],
  'm': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
  'p': [0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
  's': [0, 0, 1, 2, 2, 2, 2, 2, 3, 4, 4, 4]},
 {'$': 1, 'i': 4, 'm': 1, 'p': 2, 's': 4})

In [215]:
bwtViaSa('mississippi$')

[('$', 11), ('i$', 10), ('ippi$', 7), ('issippi$', 4), ('ississippi$', 1), ('mississippi$', 0), ('pi$', 9), ('ppi$', 8), ('sippi$', 6), ('sissippi$', 3), ('ssippi$', 5), ('ssissippi$', 2)]


'ipssm$pissii'

In [216]:
countMatchesAlex( bwtViaSa('mississippi$'), 'iss')

[('$', 11), ('i$', 10), ('ippi$', 7), ('issippi$', 4), ('ississippi$', 1), ('mississippi$', 0), ('pi$', 9), ('ppi$', 8), ('sippi$', 6), ('sissippi$', 3), ('ssippi$', 5), ('ssissippi$', 2)]
('F:', {'i': (1, 5), 'p': (6, 8), 's': (8, 12), 'm': (5, 6), '$': (0, 1)})
12


(3, 5)

## Experiments over paper BWT - Iqbal (http://biorxiv.org/content/early/2016/07/25/059170)

In [150]:
prg = 'CAAGG5CTAT6TTATTT6C5ACCT7A8G7CT$'
def toNumeric(s):
    slist = []
    for i in xrange(len(s)):
        if s[i] == 'A':
            slist.append(str(1))
        elif s[i] == 'C':
            slist.append(str(2))
        elif s[i] == 'G':
            slist.append(str(3))
        elif s[i] == 'T':
            slist.append(str(4))
        else:
            slist.append(s[i])
    return ''.join(slist)

bwm(toNumeric(prg))

['$2113352414644144462512247183724',
 '113352414644144462512247183724$2',
 '12247183724$21133524146441444625',
 '13352414644144462512247183724$21',
 '144462512247183724$2113352414644',
 '14644144462512247183724$21133524',
 '183724$2113352414644144462512247',
 '2113352414644144462512247183724$',
 '2247183724$211335241464414446251',
 '24$21133524146441444625122471837',
 '2414644144462512247183724$211335',
 '247183724$2113352414644144462512',
 '2512247183724$211335241464414446',
 '3352414644144462512247183724$211',
 '352414644144462512247183724$2113',
 '3724$211335241464414446251224718',
 '4$211335241464414446251224718372',
 '4144462512247183724$211335241464',
 '414644144462512247183724$2113352',
 '44144462512247183724$21133524146',
 '44462512247183724$21133524146441',
 '4462512247183724$211335241464414',
 '462512247183724$2113352414644144',
 '4644144462512247183724$211335241',
 '47183724$21133524146441444625122',
 '512247183724$2113352414644144462',
 '52414644144462512247183724$21133',
 

In [223]:
rankAll, tots = rankAllBwt(toNumeric(prg))
first = firstCol(tots)
first

{'$': (0, 1),
 '1': (1, 7),
 '2': (7, 13),
 '3': (13, 16),
 '4': (16, 25),
 '5': (25, 27),
 '6': (27, 29),
 '7': (29, 31),
 '8': (31, 32)}

In [151]:
def invToNumeric(s):
    slist = []
    for i in xrange(len(s)):
        if s[i] == str(1):
            slist.append('A')
        elif s[i] == str(2):
            slist.append('C')
        elif s[i] == str(3):
            slist.append('G')
        elif s[i] == str(4):
            slist.append('T')
        else:
            slist.append(s[i])
    return ''.join(slist)

In [236]:
map(lambda x: invToNumeric(x[0]), suffixArrayAll(toNumeric(prg)))

['$',
 'AAGG5CTAT6TTATTT6C5ACCT7A8G7CT$',
 'ACCT7A8G7CT$',
 'AGG5CTAT6TTATTT6C5ACCT7A8G7CT$',
 'ATTT6C5ACCT7A8G7CT$',
 'AT6TTATTT6C5ACCT7A8G7CT$',
 'A8G7CT$',
 'CAAGG5CTAT6TTATTT6C5ACCT7A8G7CT$',
 'CCT7A8G7CT$',
 'CT$',
 'CTAT6TTATTT6C5ACCT7A8G7CT$',
 'CT7A8G7CT$',
 'C5ACCT7A8G7CT$',
 'GG5CTAT6TTATTT6C5ACCT7A8G7CT$',
 'G5CTAT6TTATTT6C5ACCT7A8G7CT$',
 'G7CT$',
 'T$',
 'TATTT6C5ACCT7A8G7CT$',
 'TAT6TTATTT6C5ACCT7A8G7CT$',
 'TTATTT6C5ACCT7A8G7CT$',
 'TTT6C5ACCT7A8G7CT$',
 'TT6C5ACCT7A8G7CT$',
 'T6C5ACCT7A8G7CT$',
 'T6TTATTT6C5ACCT7A8G7CT$',
 'T7A8G7CT$',
 '5ACCT7A8G7CT$',
 '5CTAT6TTATTT6C5ACCT7A8G7CT$',
 '6C5ACCT7A8G7CT$',
 '6TTATTT6C5ACCT7A8G7CT$',
 '7A8G7CT$',
 '7CT$',
 '8G7CT$']

In [240]:
for s, t, c in zip(map(invToNumeric, bwm(toNumeric(prg))), 
                   map(lambda x: x[1], suffixArrayAll(toNumeric(prg))),
                    map(lambda x: invToNumeric(x[0]), suffixArrayAll(toNumeric(prg)))):
    print ("{0}\t{1}\t{2}".format(s,t,c))

#print('\n'.join(map(invToNumeric, bwm(toNumeric(prg)))))

$CAAGG5CTAT6TTATTT6C5ACCT7A8G7CT	31	$
AAGG5CTAT6TTATTT6C5ACCT7A8G7CT$C	1	AAGG5CTAT6TTATTT6C5ACCT7A8G7CT$
ACCT7A8G7CT$CAAGG5CTAT6TTATTT6C5	20	ACCT7A8G7CT$
AGG5CTAT6TTATTT6C5ACCT7A8G7CT$CA	2	AGG5CTAT6TTATTT6C5ACCT7A8G7CT$
ATTT6C5ACCT7A8G7CT$CAAGG5CTAT6TT	13	ATTT6C5ACCT7A8G7CT$
AT6TTATTT6C5ACCT7A8G7CT$CAAGG5CT	8	AT6TTATTT6C5ACCT7A8G7CT$
A8G7CT$CAAGG5CTAT6TTATTT6C5ACCT7	25	A8G7CT$
CAAGG5CTAT6TTATTT6C5ACCT7A8G7CT$	0	CAAGG5CTAT6TTATTT6C5ACCT7A8G7CT$
CCT7A8G7CT$CAAGG5CTAT6TTATTT6C5A	21	CCT7A8G7CT$
CT$CAAGG5CTAT6TTATTT6C5ACCT7A8G7	29	CT$
CTAT6TTATTT6C5ACCT7A8G7CT$CAAGG5	6	CTAT6TTATTT6C5ACCT7A8G7CT$
CT7A8G7CT$CAAGG5CTAT6TTATTT6C5AC	22	CT7A8G7CT$
C5ACCT7A8G7CT$CAAGG5CTAT6TTATTT6	18	C5ACCT7A8G7CT$
GG5CTAT6TTATTT6C5ACCT7A8G7CT$CAA	3	GG5CTAT6TTATTT6C5ACCT7A8G7CT$
G5CTAT6TTATTT6C5ACCT7A8G7CT$CAAG	4	G5CTAT6TTATTT6C5ACCT7A8G7CT$
G7CT$CAAGG5CTAT6TTATTT6C5ACCT7A8	27	G7CT$
T$CAAGG5CTAT6TTATTT6C5ACCT7A8G7C	30	T$
TATTT6C5ACCT7A8G7CT$CAAGG5CTAT6T	12	TATTT6C5ACCT7A8G7CT$
TAT6TTATTT6C5ACCT7A8G7CT$CAAGG5C	7	TA

In [310]:
def range_search_2d (bw, l, r, lowerSizeAlph=5, highSizeAlph=10):
    M = []
    i=l
    while (i < r):
        c = bw[i]
        if c == '$':
            c = 0
        if int(lowerSizeAlph) <= int(c) <= int(highSizeAlph):
            M.append((i, c))
        i=i+1
    return M
            
        


def countMatches(bw, p, SA):
    """
    Given BWT(T) and a pattern string p, return the number of times p occurs in T.
    """
    rankAll, tots = rankAllBwt(bw)
    first = firstCol(tots)
    sizeAlph = 8
    if p[-1] not in first:
        return 0 # character doesnt occur in T
    l, r = first[p[-1]]
    i = len(p)-2
    SA_int = [(l,r)]
    Extra_int = []
    print "F:", first
    while (i >= 0 and len(SA_int)!=0):
        print l,r
        for (l, r) in SA_int:
            print "L:",l,"R:",r
            M = range_search_2d (bw, l, r)
            for idx, num in M:
                print ("idx:", idx, "num:", num)
                if int(num)%2 == 0:
                    odd_num = str(int(num)-1)
                else:
                    odd_num = str(num)
                if SA[first[odd_num][0]] < SA[first[odd_num][0]+1]:
                    start_site = first[odd_num][0]
                    end_site = first[odd_num][0]+1
                else:
                    start_site = first[odd_num][0]+1
                    end_site = first[odd_num][0]
                print ("start_site:", start_site, "end_site:", end_site)
                if (int(num)%2 == 1) and (SA[idx] == SA[end_site] + 1):
                    end_inter = int(num)+2
                    if end_inter > sizeAlph:
                        end_inter = sizeAlph  
                    Extra_int.append ( (first[num][0], first[str(end_inter)][0]) )
                else:
                    Extra_int.append ( (first[str(start_site)][0], first[str(start_site)][0]+1))
        i = i-1
        SA_int.extend(Extra_int)
        for l,r in SA_int:
            c = p[i]
            l = first[c][0] + rankAll[c][l-1]
            r = first[c][0] + rankAll[c][r-1]
#            i -= 1
    return l, r

In [311]:
bwt_prg = bwtFromSa(toNumeric(prg))[0]
#p = 'GTTATTTAC'
p = '4412'
countMatches(bwt_prg, p, suffixArray(toNumeric(prg)))

F: {'$': (0, 1), '1': (1, 7), '3': (13, 16), '2': (7, 13), '5': (25, 27), '4': (16, 25), '7': (29, 31), '6': (27, 29), '8': (31, 32)}
7 13
L: 7 R: 13
('idx:', 9, 'num:', '7')
('start_site:', 29, 'end_site:', 30)
('idx:', 10, 'num:', '5')
('start_site:', 26, 'end_site:', 25)
('idx:', 12, 'num:', '6')
('start_site:', 26, 'end_site:', 25)


KeyError: '26'

In [284]:
suffixArray(bwt_prg)

[7,
 31,
 23,
 13,
 20,
 3,
 8,
 24,
 25,
 16,
 1,
 11,
 18,
 30,
 26,
 14,
 22,
 0,
 17,
 29,
 21,
 28,
 27,
 4,
 5,
 2,
 10,
 12,
 19,
 6,
 9,
 15]

In [ ]:
range_search_2d (bwtFromSs(toNumeric(prg)))

In [243]:
def suffixArray(s):
    ''' Given T return suffix array SA(T).  Uses "sorted"
        function for simplicity, which is probably very slow. '''
    satups = sorted([(s[i:], i) for i in xrange(0, len(s))])
    return map(lambda x: x[1], satups) # extract, return just offsets

def bwtFromSa(t, sa=None):
    ''' Given T, returns BWT(T) by way of the suffix array. '''
    bw = []
    dollarRow = None
    if sa is None:
        sa = suffixArray(t)
    for si in sa:
        if si == 0:
            dollarRow = len(bw)
            bw.append('$')
        else:
            bw.append(t[si-1])
    return (''.join(bw), dollarRow) # return string-ized version of list bw

In [159]:
class FmCheckpoints(object):
    ''' Manages rank checkpoints and handles rank queries, which are
        O(1) time, with the checkpoints taking O(m) space, where m is
        length of text. '''
    
    def __init__(self, bw, cpIval=4):
        ''' Scan BWT, creating periodic checkpoints as we go '''
        self.cps = {}        # checkpoints
        self.cpIval = cpIval # spacing between checkpoints
        tally = {}           # tally so far
        # Create an entry in tally dictionary and checkpoint map for
        # each distinct character in text
        for c in bw:
            if c not in tally:
                tally[c] = 0
                self.cps[c] = []
        # Now build the checkpoints
        for i in xrange(0, len(bw)):
            tally[bw[i]] += 1 # up to *and including*
            if (i % cpIval) == 0:
                for c in tally.iterkeys():
                    self.cps[c].append(tally[c])
    
    def rank(self, bw, c, row):
        ''' Return # c's there are in bw up to and including row '''
        if row < 0 or c not in self.cps:
            return 0
        i, nocc = row, 0
        # Always walk to left (up) when calculating rank
        while (i % self.cpIval) != 0:
            if bw[i] == c:
                nocc += 1
            i -= 1
        return self.cps[c][i // self.cpIval] + nocc

In [160]:
st = 'teststring'
#     0123456789
cps = FmCheckpoints(st)

In [161]:
# should get back list of integers, where elt i gives
# # times 't' appears up to and including offset i
[ cps.rank(st, 't', i) for i in xrange(10) ]

[1, 1, 1, 2, 2, 3, 3, 3, 3, 3]

In [163]:
# likewise for 'g'
[ cps.rank(st, 'g', i) for i in xrange(10) ]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [164]:
class FmIndex():
    ''' O(m) size FM Index, where checkpoints and suffix array samples are
        spaced O(1) elements apart.  Queries like count() and range() are
        O(n) where n is the length of the query.  Finding all k
        occurrences of a length-n query string takes O(n + k) time.
        
        Note: The spacings in the suffix array sample and checkpoints can
        be chosen differently to achieve different bounds. '''
    
    @staticmethod
    def downsampleSuffixArray(sa, n=4):
        ''' Take only the suffix-array entries for every nth suffix.  Keep
            suffixes at offsets 0, n, 2n, etc with respect to the text.
            Return map from the rows to their suffix-array values. '''
        ssa = {}
        for i in xrange(0, len(sa)):
            # We could use i % n instead of sa[i] % n, but we lose the
            # constant-time guarantee for resolutions
            if sa[i] % n == 0:
                ssa[i] = sa[i]
        return ssa
    
    def __init__(self, t, cpIval=4, ssaIval=4):
        if t[-1] != '$':
            t += '$' # add dollar if not there already
        # Get BWT string and offset of $ within it
        sa = suffixArray(t)
        self.bwt, self.dollarRow = bwtFromSa(t, sa)
        # Get downsampled suffix array, taking every 1 out of 'ssaIval'
        # elements w/r/t T
        self.ssa = self.downsampleSuffixArray(sa, ssaIval)
        self.slen = len(self.bwt)
        # Make rank checkpoints
        self.cps = FmCheckpoints(self.bwt, cpIval)
        # Calculate # occurrences of each character
        tots = dict()
        for c in self.bwt:
            tots[c] = tots.get(c, 0) + 1
        # Calculate concise representation of first column
        self.first = {}
        totc = 0
        for c, count in sorted(tots.iteritems()):
            self.first[c] = totc
            totc += count
    
    def count(self, c):
        ''' Return number of occurrences of characters < c '''
        if c not in self.first:
            # (Unusual) case where c does not occur in text
            for cc in sorted(self.first.iterkeys()):
                if c < cc: return self.first[cc]
            return self.first[cc]
        else:
            return self.first[c]
    
    def range(self, p):
        ''' Return range of BWM rows having p as a prefix '''
        l, r = 0, self.slen - 1 # closed (inclusive) interval
        for i in xrange(len(p)-1, -1, -1): # from right to left
            l = self.cps.rank(self.bwt, p[i], l-1) + self.count(p[i])
            r = self.cps.rank(self.bwt, p[i], r)   + self.count(p[i]) - 1
            if r < l:
                break
        return l, r+1
    
    def resolve(self, row):
        ''' Given BWM row, return its offset w/r/t T '''
        def stepLeft(row):
            ''' Step left according to character in given BWT row '''
            c = self.bwt[row]
            return self.cps.rank(self.bwt, c, row-1) + self.count(c)
        nsteps = 0
        while row not in self.ssa:
            row = stepLeft(row)
            nsteps += 1
        return self.ssa[row] + nsteps
    
    def hasSubstring(self, p):
        ''' Return true if and only if p is substring of indexed text '''
        l, r = self.range(p)
        return r > l
    
    def hasSuffix(self, p):
        ''' Return true if and only if p is suffix of indexed text '''
        l, r = self.range(p)
        off = self.resolve(l)
        return r > l and off + len(p) == self.slen-1
    
    def occurrences(self, p):
        ''' Return offsets for all occurrences of p, in no particular order '''
        l, r = self.range(p)
        return [ self.resolve(x) for x in xrange(l, r) ]

In [165]:
fm = FmIndex('abaaba')

In [166]:
fm.hasSubstring('aaba')

True

In [168]:
fm.hasSubstring('aabb')

False

In [169]:
fm.range('a')

(1, 5)

In [170]:
fm.range('baaba')

(6, 7)

In [171]:
p, t = "CAT", "TTGTGTGCATGTTGTTTCATCATTTAGAGATACATTGCGCTGCATCATGGTCA"
#              01234567890123456789012345678901234567890123456789012
# Occurrences:        *         *  *           *         *  *
fm = FmIndex(t)
matches = sorted(fm.occurrences(p))
matches == [7, 17, 20, 32, 42, 45]

True